In [11]:
import pandas as pd
import numpy as np
import pydeck as pdk

In [12]:
df = pd.read_csv("../data/sample_taxi.csv")

In [13]:
df.head()

,taxi_id,time,lon,lat,is_passenger,speed
0,22224,10:13:42,113.887032,22.547518,0,0
1,22224,19:31:36,113.909554,22.582199,1,71
2,22224,17:06:32,113.916748,22.623550,1,100
3,22224,15:07:19,113.922432,22.535383,1,21
4,22224,14:55:18,113.922668,22.520033,1,10


In [14]:
len(df)

9843421

In [15]:
# layer = pydeck.Layer(
#     'HexagonLayer',
#     df,
#     get_position=['lon', 'lat'],
#     auto_highlight=True,
#     elevation_scale=50,
#     pickable=True,
#     elevation_range=[0, 3000],
#     extruded=True,
#     coverage=1)

In [16]:
df.info()
# df['lon'] = df['lon'].round(4)
# df['lat'] = df['lat'].round(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9843421 entries, 0 to 9843420
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   taxi_id       int64  
 1   time          object 
 2   lon           float64
 3   lat           float64
 4   is_passenger  int64  
 5   speed         int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 450.6+ MB


In [21]:
# df2 = df.head(50000)
df.describe()

,taxi_id,lon,lat,is_passenger,speed
count,9.843421e+06,9.843421e+06,9.843421e+06,9.843421e+06,9.843421e+06
mean,2.944999e+04,1.140428e+02,2.266443e+01,4.322090e-01,2.027840e+01
std,4.381604e+03,8.802736e-01,1.361056e+02,4.953831e-01,2.411174e+01
min,2.222400e+04,1.080019e+02,1.028000e+01,0.000000e+00,0.000000e+00
25%,2.564400e+04,1.139622e+02,2.254113e+01,0.000000e+00,0.000000e+00
50%,2.923700e+04,1.140498e+02,2.256608e+01,0.000000e+00,1.100000e+01
75%,3.328600e+04,1.141124e+02,2.262542e+01,1.000000e+00,3.600000e+01
max,3.694700e+04,9.926038e+02,3.402800e+05,1.000000e+00,1.200000e+02


In [24]:
df2 = df[df['lat']>22]

In [25]:
df2 = df2[df2['lat']<23]

In [26]:
df2 = df2[df2['lon']>113]

In [28]:
df2 = df2[df2['lon']<115]

In [29]:
df2.describe()

,taxi_id,lon,lat,is_passenger,speed
count,9.834359e+06,9.834359e+06,9.834359e+06,9.834359e+06,9.834359e+06
mean,2.945006e+04,1.140412e+02,2.258918e+01,4.321136e-01,2.025592e+01
std,4.381736e+03,1.091883e-01,6.386863e-02,4.953700e-01,2.408338e+01
min,2.222400e+04,1.132646e+02,2.221920e+01,0.000000e+00,0.000000e+00
25%,2.564400e+04,1.139623e+02,2.254113e+01,0.000000e+00,0.000000e+00
50%,2.923700e+04,1.140499e+02,2.256602e+01,0.000000e+00,1.100000e+01
75%,3.328600e+04,1.141124e+02,2.262530e+01,1.000000e+00,3.600000e+01
max,3.694700e+04,1.149990e+02,2.299996e+01,1.000000e+00,1.200000e+02


In [31]:
data = df2
data.columns = ['taxi_id', 'time', 'lon', 'lat', 'OpenStatus', 'Speed']
data['coordinates'] = data.apply(lambda r: [r['lon'], r['lat']], axis=1)
data['timestamp'] = data['time'].str.slice(0, 2).astype('int') * 3600 + data['time'].str.slice(3, 5).astype('int') * 60 + data['time'].str.slice(6, 8).astype('int') * 1
data = data.sort_values(by=['taxi_id', 'timestamp'])
data['coordinates'] = data['coordinates'].apply(lambda r: [r])
data['timestamp'] = data['timestamp'].apply(lambda r: [r])
df = data.groupby('taxi_id')['coordinates', 'timestamp'].sum()
df = df.reset_index()

<ipython-input-31-dc41e6572ccd>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = data.groupby('taxi_id')['coordinates', 'timestamp'].sum()


In [34]:
df.columns = ['vendor', 'coordinates', 'timestamp']
import random
color = []
for i in range(0, 3000):
    iop = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
    color.append(iop)
df['color'] = color
df.to_json('./file4.json', orient='records', lines=True)

In [18]:
# layer = pdk.Layer(
#      #'HexagonLayer',
#     'TripsLayer',
#     df2,                                   #数据在此输入
#     get_position=['lon', 'lat'],                #指定经纬度的列
#     getColorWeight = 'taxi_id',                #指定集计数量的列
#     getElevationWeight = 'taxi_id',            #指定集计数量的列
#     getTimeStamps='timestamps'
#      auto_highlight=True,
#      elevation_scale=10,
#      pickable=True,
#      radius = 200,                                #Hexagon的大小
#      elevation_range=[0, 500],                    #Hexagon的高低
#      extruded=True,
#      coverage=1)                                  #Hexagon覆盖比例
#
# # Set the viewport location
# view_state = pdk.ViewState(
#     longitude=114.027465,                         #地图底图中心经纬度
#     latitude=22.632468,                           #地图底图中心经纬度
#     zoom=10,                                      #地图底图缩放等级
#     min_zoom=5,
#     max_zoom=12,
#     pitch=40,                                     #地图底图俯视角度
#     bearing=-10)                                  #地图底图选转角度
#
# # Render
# r = pdk.Deck(layers=layer,                       #创建deckgl实例
#              initial_view_state=view_state,
#              api_keys = {'mapbox':'sk.eyJ1IjoibGV0aGFsMjMzIiwiYSI6ImNrbzU3Y3JxMDE5aGUyb3M1dnY3c20zajIifQ.guC2wc_8Sx7ZuegWEshs5g'},               #你的mapbox_key
#              map_style =  'mapbox://styles/mapbox/dark-v9'#mapbox底图的风格
#             )
# r.show()

In [19]:
# layer = pdk.Layer(
#      'HeatmapLayer',
#     df.head(100000),                                   #数据在此输入
#     get_position=['lon', 'lat'],                #指定经纬度的列
#     getWeight = 'taxi_id',                     #指定权重
#      )
#
# # Set the viewport location
# view_state = pdk.ViewState(
#     longitude=114.027465,
#     latitude=22.632468,
#     zoom=10,
#     min_zoom=5,
#     max_zoom=12,
#     pitch=40,
#     bearing=-10)
#
# # Render
# r = pdk.Deck(layers=[layer],
#              initial_view_state=view_state,
#              api_keys = {'mapbox':'sk.eyJ1IjoibGV0aGFsMjMzIiwiYSI6ImNrbzU3Y3JxMDE5aGUyb3M1dnY3c20zajIifQ.guC2wc_8Sx7ZuegWEshs5g'},
#               map_style =  'mapbox://styles/mapbox/dark-v9'
#             )
# r.show()

In [20]:
# layer = pdk.Layer(
#      'ScreenGridLayer',
#     df.head(100000),                                   #数据在此输入
#     get_position=['lon', 'lat'],                #指定经纬度的列
#     getWeight = 'taxi_id',                     #指定权重
#      opacity =  1,
#     cellSizePixels = 10,
#     colorRange = [
#         [255,255,204,25],
#         [199,233,180,85],
#         [127,205,187,127],
#         [65,182,196,170],
#         [44,127,184,190],
#         [37,52,148,255]
#         ],
#      )
#
# # Set the viewport location
# view_state = pdk.ViewState(
#     longitude=114.027465,
#     latitude=22.632468,
#     zoom=10,
#     min_zoom=5,
#     max_zoom=16,
#     pitch=40,
#     bearing=-10)
#
# # Render
# r = pdk.Deck(layers=[layer],
#              initial_view_state=view_state,
#              api_keys = {'mapbox':'sk.eyJ1IjoibGV0aGFsMjMzIiwiYSI6ImNrbzU3Y3JxMDE5aGUyb3M1dnY3c20zajIifQ.guC2wc_8Sx7ZuegWEshs5g'},
#               map_style =  'mapbox://styles/mapbox/dark-v9'
#             )
# r.show()